In [1]:
# coding: utf-8

import random
import pygame
import sys
from pygame.locals import *
import numpy as np

speed = 12
FrameWidth = 800
FrameHeight = 500
Grid_Size = 20  # Width and height of the Grids
# Ensuring that the Grids fit perfectly in the window. eg if Grid size was
# 10     and window width or windowheight were 15 only 1.5 Grids would
# fit.
# assert FrameWidth % Grid_Size == 0, "Window width must be a multiple of Grid size."
# # Ensuring that only whole integer number of Grids fit perfectly in the window.
# assert FrameHeight % Grid_Size == 0, "Window height must be a multiple of Grid size."
Grid_W = int(FrameWidth / Grid_Size)  # Grid Width
Grid_H = int(FrameHeight / Grid_Size)  # Grids Height


White = (255, 255, 255)
Black = (0, 0, 0)
Red = (255, 0, 0)  # Defining element colors for the program.
Gold = (255, 215, 0)
Blue = (0, 0, 255)
Gray = (40, 40, 40)
RedPink = (249, 91, 91)

BackgroundCOLOR = Gold  # Background color

apple_count = 10 # see below, under rungame, there's a for loop for this


UP = 'up'
DOWN = 'down'      # Defining keyboard keys.
LEFT = 'left'
RIGHT = 'right'

HEAD = 0  # Syntactic sugar: index of the snake's head; use for defining that the head starts from the first one


# In[ ]:


def main():
    global Snakespeed_CLOCK, Surfingdisplay, FONT_BASIC

    pygame.init()
    Snakespeed_CLOCK = pygame.time.Clock()
    Surfingdisplay = pygame.display.set_mode((FrameWidth, FrameHeight))
    FONT_BASIC = pygame.font.Font('freesansbold.ttf', 18)
    pygame.display.set_caption('Snake')

    StartScreen()
    while True:
        runGame()
        EndScreen()


def runGame():
    #Set a random start point.
    start_x = random.randint(5, Grid_W - 6)
    start_y = random.randint(5, Grid_H - 6)
    Coords_worm = [{'x': start_x, 'y': start_y},
                  {'x': start_x - 1, 'y': start_y},
                  {'x': start_x - 2, 'y': start_y}]
    direction = RIGHT

    # define apple numbers and locations
    apple_list = []
    # Start the apple in a random place.
    for i in range(apple_count):
        apple_list.append(getRandomLocation())


    while True:  # main game loop
        for event in pygame.event.get():  # event handling loop
            if event.type == QUIT:
                terminate()
            elif event.type == KEYDOWN:
                # left key
                if (event.key == K_LEFT) and direction != RIGHT:
                    direction = LEFT
                # right key
                elif (event.key == K_RIGHT) and direction != LEFT:
                    direction = RIGHT
                # up key
                elif (event.key == K_UP) and direction != DOWN:
                    direction = UP
                # down key
                elif (event.key == K_DOWN) and direction != UP:
                    direction = DOWN
                # space key!!!! 加速，move五个格子
                elif (event.key == K_SPACE):
                    if direction == UP:
                        new_Coords_worm = []
                        for i in range(len(Coords_worm)):
                            new_coordination = {'x': Coords_worm[i]['x'],
                                       'y': Coords_worm[i]['y'] - 3}
                            new_Coords_worm.append(new_coordination)
                    if direction == DOWN:
                        new_Coords_worm = []
                        for i in range(len(Coords_worm)):
                            new_coordination = {'x': Coords_worm[i]['x'],
                                       'y': Coords_worm[i]['y'] + 3}
                            new_Coords_worm.append(new_coordination)
                    if direction == LEFT:
                        new_Coords_worm = []
                        for i in range(len(Coords_worm)):
                            new_coordination = {'x': Coords_worm[i]['x'] - 3,
                                       'y': Coords_worm[i]['y']}
                            new_Coords_worm.append(new_coordination)
                    if direction == RIGHT:
                        new_Coords_worm = []
                        for i in range(len(Coords_worm)):
                            new_coordination = {'x': Coords_worm[i]['x'] + 3,
                                       'y': Coords_worm[i]['y']}
                            new_Coords_worm.append(new_coordination)
                    Coords_worm = new_Coords_worm
                # escape key, quit
                elif event.key == K_ESCAPE:
                    terminate()

         # check if the Snake has hit itself or the edge
        if Coords_worm[HEAD]['x'] == -1 or Coords_worm[HEAD]['x'] == Grid_W or Coords_worm[HEAD]['y'] == -1 or Coords_worm[HEAD]['y'] == Grid_H:
            print("!") # check whether this step has a bug, if 这个被print出来了，那么证明这个是正确的，游戏结束就是因为这个步骤
            return  # game over wall
        # if worm's body hits head
        for wormBody in Coords_worm[1:]:
            if wormBody['x'] == Coords_worm[HEAD]['x'] and wormBody['y'] == Coords_worm[HEAD]['y']:
                print("?")
                return  # game over body to head

        # check if Snake has eaten an apple
        apple_eaten_list = [Coords_worm[HEAD]['x'] == apple_list[i]['x'] 
                           and Coords_worm[HEAD]['y'] == apple_list[i]['y'] 
                           for i in range(len(apple_list))]
        if sum(apple_eaten_list) == 1: # 吃到了一个苹果
        # 0=false, 1=true
        # if wormCoords[HEAD]['x'] == apple['x'] and wormCoords[HEAD]['y'] == apple['y']:
            # don't remove worm's tail segment
            i = np.argmax(apple_eaten_list)
            del apple_list[i] # 这个被吃掉的苹果被删除了
            apple_list.append(getRandomLocation())  # set a new apple somewhere
        else:
            del Coords_worm[-1]  # remove worm's tail segment，因为每走一步，都是头多一个，尾巴多一个，所以正常的尾巴-1

        # move the worm by adding a segment in the direction it is moving，头 + 1
        if direction == UP:
            newHead = {'x': Coords_worm[HEAD]['x'],
                       'y': Coords_worm[HEAD]['y'] - 1}
        elif direction == DOWN:
            newHead = {'x': Coords_worm[HEAD]['x'],
                       'y': Coords_worm[HEAD]['y'] + 1}
        elif direction == LEFT:
            newHead = {'x': Coords_worm[HEAD]['x'] - 1, 
                       'y': Coords_worm[HEAD]['y']}
        elif direction == RIGHT:
            newHead = {'x': Coords_worm[HEAD]['x'] + 1, 
                       'y': Coords_worm[HEAD]['y']}
        Coords_worm.insert(0, newHead)
        Surfingdisplay.fill(BackgroundCOLOR)
        drawGrid()
        drawWorm(Coords_worm)
        for i in range(len(apple_list)):
            drawApple(apple_list[i])
        drawScore(len(Coords_worm) - 3)
        pygame.display.update()
        Snakespeed_CLOCK.tick(speed)

def drawingKeyMsgbutton():
    Surfingkey_button = FONT_BASIC.render('Press any key to start!', True, RedPink)
    KeyRect_button = Surfingkey_button.get_rect(topleft=(FrameWidth - 200, FrameHeight - 30))
    Surfingdisplay.blit(Surfingkey_button, KeyRect_button)


def CheckingKeyPress():
    if len(pygame.event.get(QUIT)) > 0:
        terminate()
    keyUp_showing = pygame.event.get(KEYUP)
    if len(keyUp_showing) == 0:
        return None
    if keyUp_showing[0].key == K_ESCAPE:
        terminate()
    return keyUp_showing[0].key


def StartScreen():
    Font_title = pygame.font.Font('freesansbold.ttf', 60)
    startSurf = Font_title.render("Let's Play Greedy Snake!", True, RedPink) 
    Surfingdisplay.fill(BackgroundCOLOR) 
    startRect = startSurf.get_rect(center=(FrameWidth/2, FrameHeight/2)) 
    Surfingdisplay.blit(startSurf, startRect) 
    drawingKeyMsgbutton()
    pygame.display.update()
    
    while True:
        if CheckingKeyPress():
            pygame.event.get()  # clear event queue
            return
    

def terminate():
    pygame.quit()
    sys.exit()


def getRandomLocation():
    return {'x': random.randint(0, Grid_W - 1), 'y': random.randint(0, Grid_H - 1)}


def EndScreen():
    Font_gameOver = pygame.font.Font('freesansbold.ttf', 100)
    Surfing1 = Font_gameOver.render('You', True, White) #
    Surfing2 = Font_gameOver.render('Lose', True, White) #
    gameRect = Surfing1.get_rect(midtop=(FrameWidth / 2, 10)) #
    overRect = Surfing2.get_rect(midtop=(FrameWidth / 2, gameRect.height + 30)) #

    Surfingdisplay.blit(Surfing1, gameRect)
    Surfingdisplay.blit(Surfing2, overRect)
    drawingKeyMsgbutton()
    pygame.display.update()
    pygame.time.wait(500)
    CheckingKeyPress()  
    
    while True:
        if CheckingKeyPress():
            pygame.event.get()  
            return

def drawScore(score):
    scoreSurfing = FONT_BASIC.render('Score: %s' % (score), True, Red)
    scoreRect = scoreSurfing.get_rect(topleft =(FrameWidth-120, 10))
    Surfingdisplay.blit(scoreSurfing, scoreRect)

def drawWorm(Coords_worm):
    for coord in Coords_worm:
        x = coord['x'] * Grid_Size
        y = coord['y'] * Grid_Size
        wormSegmentRect = pygame.Rect(x, y, Grid_Size, Grid_Size)
        pygame.draw.rect(Surfingdisplay, Black, wormSegmentRect)
        wormInnerSegmentRect = pygame.Rect(x + 4, y + 4, Grid_Size - 8, Grid_Size - 8)
        pygame.draw.rect(Surfingdisplay, Black, wormInnerSegmentRect)

def drawApple(coord):
    x = coord['x'] * Grid_Size
    y = coord['y'] * Grid_Size
    appleRect = pygame.Rect(x, y, Grid_Size, Grid_Size)
    pygame.draw.rect(Surfingdisplay, Red, appleRect)
    
def drawGrid():
    for x in range(0, FrameWidth, Grid_Size):  # draw vertical lines
        pygame.draw.line(Surfingdisplay, Gray, (x, 0), (x, FrameHeight))
    for y in range(0, FrameHeight, Grid_Size):  # draw horizontal lines
        pygame.draw.line(Surfingdisplay, Gray, (0, y), (FrameWidth, y))


if __name__ == '__main__':
    try:
        main()
    except SystemExit:
        pass


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
!
